In [ ]:
%%time
from keras.utils import np_utils
from keras.callbacks import ModelCheckpoint
import wave
import numpy
from music21 import converter, instrument, note, chord
import glob
import os
import Paths
from pydub import AudioSegment
from keras.models import Sequential
from keras.layers import Dense, LSTM, MaxPooling2D, Dropout, Activation
import librosa
import tensorflow as tf
from keras.layers import LeakyReLU

directory = os.listdir(Paths.IN_DIRECTORY)
array = []

sequence_length = []
for sample_name in directory:
    if 'wav' in sample_name:
        file_path = os.path.join(Paths.IN_DIRECTORY, sample_name)
        song, sr = librosa.load(file_path)
        sequence_length.append(len(song))
        #print(len(song))
        #song = AudioSegment.from_wav(file_path)
        #samples = song.get_array_of_samples()
        array.append(song)

# Removing the last element using slicing
lastElementIndex = len(array)-1
in_array = array[:lastElementIndex]
out_array = array[1:]
'''
# Create flat list of Input and Output
Input_flat=[]
for sublist in In_array:
    for element in sublist:
        Input_flat.append(element)

Output_flat=[]
for sublist in Out_array:
    for element in sublist:
        Output_flat.append(element)


# create output sequence for samples following input
output = []
for i in range(0, len(array)-1):
 i+=1
 #print(i)
 output.append(array[i])
'''
# output_stack = numpy.stack(out_array)

# Get overall number of Input values from samples
#Number_of_values = sum(x for x in sequence_length)


# Create the Input and Output to tensors
input_tensor = tf.convert_to_tensor(in_array)
output_tensor = tf.convert_to_tensor(out_array)

batch_size = len(input_tensor)
timestep = sequence_length[0]

input_data = tf.reshape(input_tensor, shape=(batch_size, timestep, 1))
output_data = tf.reshape(output_tensor, shape=(batch_size, timestep, 1))

# one hot encode the output variable
y = np_utils.to_categorical(output_data)

#Normalize the training data based on overall number of values
#input_data = input_data / Number_of_values

model = Sequential()
first_lstm = LSTM(32,
        batch_input_shape=(batch_size, timestep, 1),
        return_sequences=True, activation='tanh',
        stateful=True)

model.add(first_lstm)
model.add(LeakyReLU())
model.add(Dropout(0.2))

#this is the last LSTM layer, use return_sequences=False
model.add(LSTM(16, return_sequences=False, stateful=True,  activation='tanh'))

model.add(Dropout(0.2))
model.add(LeakyReLU())

model.add(Dense(1))

model.compile(loss='binary_crossentropy', optimizer='rmsprop')

model.fit(input_data, output_data, epochs=2, batch_size=sequence_length[0], verbose=2)

'''

model = Sequential()
#model.add(LSTM(units= 1, batch_input_shape= [batch_size, timestep, 1], stateful=True, return_sequences=True))
model.add(LSTM(units= 1, activation='relu', stateful=True, return_sequences=True, batch_input_shape=(batch_size, timestep, 1)))
model.compile(loss='categorical_crossentropy', optimizer='rmsprop')
model.fit(input_data, output_data, epochs=5, batch_size=sequence_length[0], verbose=2)
'''

